In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [2]:
import sklearn
from sklearn.preprocessing import StandardScaler

iris = load_iris()
X = iris['data']
y = iris['target']
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
clf = DecisionTreeClassifier(random_state=0)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.6, shuffle=True)
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best')

In [ ]:
## This will create and export the decision tree as a pdf
import graphviz 
dot_data = tree.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("iris")

In [3]:
print('left --    ',clf.tree_.children_left)
print('right --   ',clf.tree_.children_right)
print('features --',clf.tree_.feature)

left --     [ 1 -1  3  4 -1  6 -1  8 -1 10 -1 -1 13 14 -1 -1 -1]
right --    [ 2 -1 12  5 -1  7 -1  9 -1 11 -1 -1 16 15 -1 -1 -1]
features -- [ 3 -2  2  3 -2  1 -2  3 -2  0 -2 -2  0  0 -2 -2 -2]


In [4]:
## Modified for decision feature prediction model

left_nodes = clf.tree_.children_left[clf.tree_.children_left>0]
right_nodes = clf.tree_.children_right[clf.tree_.children_right>0]
node_indicator = clf.decision_path(X)
path_list = []
for i, j in enumerate(X):
    path_list.append(node_indicator.indices[node_indicator.indptr[i]:node_indicator.indptr[i+1]])

## Convert path to strings

test_df = []
# test_path = []
# test_cut_feats = []
path_column = []
cuts_feat = []
for i, j in enumerate(X):
    path_as_string = []
    for node in path_list[i]:
        if node == 0:
            path_as_string.append('S')
            for val in X[i]:
                test_df.append(val)
            test_df.append(['S', 0])
        elif node in left_nodes:
            path_as_string.append('L')
            for val in X[i]:
                test_df.append(val)
            if clf.tree_.feature[node] < 0:
                test_df.append(['L', 0])
            else:
                test_df.append(['L', clf.tree_.feature[node]+1])
        elif node in right_nodes:
            path_as_string.append('R')
            for val in X[i]:
                test_df.append(val)
            if clf.tree_.feature[node] < 0:
                test_df.append(['R', 0])
            else:
                test_df.append(['R', clf.tree_.feature[node]+1])
            
    path_as_string.append('E')
    for val in X[i]:
        test_df.append(val)
    test_df.append(['E', 0])
    # path_as_string = ' '.join(path_as_string)
    # path_column = np.append(path_column, path_as_string)
    path_column.append(path_as_string)

path_column = np.array(path_column)

In [5]:
final_test_df = pd.DataFrame(np.array(test_df).reshape(-1,5))

In [6]:
final_test_df[[5,6]] = pd.DataFrame(final_test_df[4].values.tolist(), index= final_test_df.index)

In [7]:
final_test_df.head()

,0,1,2,3,4,5,6
0,-0.900681,1.019,-1.34023,-1.31544,"[S, 0]",S,0
1,-0.900681,1.019,-1.34023,-1.31544,"[L, 0]",L,0
2,-0.900681,1.019,-1.34023,-1.31544,"[E, 0]",E,0
3,-1.14302,-0.131979,-1.34023,-1.31544,"[S, 0]",S,0
4,-1.14302,-0.131979,-1.34023,-1.31544,"[L, 0]",L,0


In [ ]:
# left_nodes = clf.tree_.children_left[clf.tree_.children_left>0]
# right_nodes = clf.tree_.children_right[clf.tree_.children_right>0]
# node_indicator = clf.decision_path(X)
# path_list = []
# for i, j in enumerate(X):
#     path_list.append(node_indicator.indices[node_indicator.indptr[i]:node_indicator.indptr[i+1]])

# ## Convert path to strings
# path_column = np.array([])
# for i, j in enumerate(X):
#     path_as_string = []
#     for node in path_list[i]:
#         if node == 0:
#             path_as_string.append('S')
#         elif node in left_nodes:
#             path_as_string.append('L')
#         elif node in right_nodes:
#             path_as_string.append('R')
            
#     path_as_string.append('E')
#     path_as_string = ' '.join(path_as_string)
#     path_column = np.append(path_column, path_as_string)

In [ ]:
chars = ['S', 'L', 'R', 'E']
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Xnew = np.hstack((X, path_column.reshape(-1,1)))
path_sequence = Xnew[:,4]
data = pd.DataFrame(Xnew)
data[5]=y
df = data.sample(frac=1).reset_index(drop=True)

# prepare dataset for training
get_path_lengths = lambda t: len(t.split())
paths_lengths = np.array([get_path_lengths(xi) for xi in path_sequence])

In [ ]:
vocab_size = 4
label_size = 3
feature_size = 4
# cut the text in semi-redundant sequences of maxlen characters
maxlen = np.max(paths_lengths)
sentences = []
next_chars = []
features = []
labels = []

for i in range(0, len(df)):
    # get the feature
    curr_feat = np.array([df.iloc[i,0:4]])
    curr_path = df.iloc[i,4].split()
    curr_path_len = len(curr_path)
    curr_label = y[i]
    for j in range(1,curr_path_len):
        features.append(curr_feat)
        labels.append(curr_label)
        sentences.append(curr_path[0:j])
        next_chars.append(curr_path[j])
print('Vectorization...')

x_sent = np.zeros((len(sentences), maxlen, vocab_size), dtype=np.bool)
x_feat = np.zeros((len(sentences), feature_size), dtype=np.float)
y_chars = np.zeros((len(sentences), vocab_size), dtype=np.bool)
y_feat = np.zeros((len(sentences), label_size), dtype=np.float)
#from keras.utils import to_categorical
#y_feat_tmp = to_categorical(df[5])

In [ ]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x_sent[i, t, char_indices[char]] = 1
    y_chars[i, char_indices[next_chars[i]]] = 1
    x_feat[i,:] = features[i]
    y_feat[i,labels[i]]=1

In [ ]:
index = 10
print(y_chars[index],y_feat[index],x_sent[index],x_feat[index])
print(y_chars.shape,y_feat.shape,x_sent.shape,x_feat.shape)

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Concatenate, concatenate, Flatten

h1_size = 5
latent_dim = 5

input_x_features = Input(shape=(feature_size,),name='ip_x')
hidden_state_x = Dense(h1_size, activation='relu',name='hidden_x')(input_x_features)
output_labels = Dense(3, activation='softmax',name='op_x')(hidden_state_x)

input_sent_features = Input(shape=(maxlen,vocab_size),name='ip_sent')
decoder = LSTM(latent_dim,return_state=False,return_sequences=False,name='lstm_sent')
decoder_outputs = decoder(input_sent_features)

merge_layer = concatenate([hidden_state_x,decoder_outputs],name='cat')
output_chars = Dense(vocab_size, activation='softmax',name='op_sent')(merge_layer)
model = Model([input_x_features,input_sent_features], [output_labels,output_chars])
model.summary()

In [ ]:
def paths_joint_model(initialize=True, rnn_cell= 'gru'):
    from keras.models import Model
    from keras.layers import Input, LSTM, Dense, Concatenate, concatenate, Flatten, GRU
    h1_size = 5
    latent_dim = 5
    
    input_x_features = Input(shape=(feature_size,),name='ip_x')
    hidden_state_x = Dense(h1_size, activation='relu',name='hidden_x')(input_x_features)
    output_labels = Dense(3, activation='softmax',name='op_x')(hidden_state_x)
    
    input_sent_features = Input(shape=(maxlen,vocab_size),name='ip_sent')
    if rnn_cell == 'gru':
        RNN = GRU
    else:
        RNN = LSTM
            
    decoder = RNN(latent_dim,return_state=False,return_sequences=False,name='lstm_sent')
    if initialize:
        decoder_outputs = decoder(input_sent_features,initial_state=hidden_state_x)
    else:
        decoder_outputs = decoder(input_sent_features)
    
    merge_layer = concatenate([hidden_state_x,decoder_outputs],name='cat')
    output_chars = Dense(vocab_size, activation='softmax',name='op_sent')(merge_layer)
    model = Model([input_x_features,input_sent_features], [output_labels,output_chars])
    return model

In [ ]:
model = paths_joint_model()
model.compile(optimizer='adam', loss={'op_x':'categorical_crossentropy','op_sent':'categorical_crossentropy'},metrics=['accuracy'])
model.fit([x_feat,x_sent],[y_feat,y_chars],batch_size =20, epochs = 2,verbose=1)

In [ ]:
def sample(x):
    n = x.shape[0]
    x_f = x.reshape(1,feature_size)
    token = 'S'
    cont = True
    text = [token]
    x_sent = np.zeros((1,maxlen,vocab_size),dtype=np.bool)
    x_sent[0,0,char_indices[token]] = 1
    label = []
    index = 1
    while cont & (index <maxlen):
        pred = model.predict([x_f.reshape(1,feature_size),x_sent])
        char_index = np.argmax(pred[1])
        label.append(np.argmax(pred[0])) 
        x_sent[0,index,char_index] = 1
        next_char = indices_char[char_index]
        text.append(next_char)
        index += 1    
        if next_char == 'E':
            cont = False
    return [text,label]

In [ ]:
count = []
for i in range(10,20):
    curr_feat = np.array([df.iloc[i,0:4]])
    path,label= sample(curr_feat)
    print('actual vs predicted: ', df.iloc[i,4] ,' vs ', ' '.join(path), 'labels: ', df.iloc[i,5],label[0])
    count.append(df.iloc[i,5]==label[0])
np.mean(count)

In [ ]:
def paths_model(initialize=True, rnn_cell= 'gru',latent_dim = 5):
    from keras.models import Model
    from keras.layers import Input, LSTM, Dense, Concatenate, concatenate, Flatten, GRU
    latent_dim = latent_dim
    
    hidden_state_x = Input(shape=(latent_dim,),name='hidden_x')
    input_sent_features = Input(shape=(maxlen,vocab_size),name='ip_sent')
    if rnn_cell == 'gru':
        RNN = GRU
    else:
        RNN = LSTM
            
    decoder = RNN(latent_dim,return_state=False,return_sequences=False,name='gru_sent')
    if initialize:
        decoder_outputs = decoder(input_sent_features,initial_state=hidden_state_x)
    else:
        decoder_outputs = decoder(input_sent_features)
    
    merge_layer = concatenate([hidden_state_x,decoder_outputs],name='cat')
    output_chars = Dense(vocab_size, activation='softmax',name='op_sent')(merge_layer)
    model = Model([hidden_state_x,input_sent_features], output_chars)
    return model

def label_model(feature_size = 4, latent_dim = 5):
    from keras.models import Model
    from keras.layers import Input, LSTM, Dense, Concatenate, concatenate, Flatten, GRU
    feature_size = feature_size
    h1_size = latent_dim
    input_x_features = Input(shape=(feature_size,),name='ip_x')
    hidden_state_x1 = Dense(20, activation='tanh',name='hidden_x1')(input_x_features)
    hidden_state_x2 = Dense(20, activation='tanh',name='hidden_x2')(hidden_state_x1)
    hidden_state_x3 = Dense(h1_size, activation='tanh',name='hidden_x3')(hidden_state_x2)
    output_labels = Dense(3, activation='softmax',name='op_x')(hidden_state_x3)    
    model = Model(input_x_features,output_labels)
    return model

from keras import backend as K

def get_hidden_x(x,model,layer_num=3):
    def get_hidden_x_inner(model,layer_num=layer_num):
        return K.function([model.layers[0].input], [model.layers[layer_num].output])
    return get_hidden_x_inner(model,layer_num=layer_num)([x])[0]

In [ ]:
path_m = paths_model()
path_m.summary()
label_m = label_model()
label_m.summary()

In [ ]:
from keras.utils import to_categorical

y_cat = to_categorical(y)

label_m.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
label_m_history = label_m.fit(X,y_cat,batch_size =20, epochs = 2,verbose=0, shuffle=True)

x_latent = get_hidden_x(x_feat,model=label_m)

path_m.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
path_m.fit([x_latent,x_sent],y_chars,batch_size =20, epochs = 2,verbose=1, shuffle=True)

In [ ]:
from sklearn.metrics import jaccard_score

latent_dim = 5

def jaccard_score_inconsistent(x,y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)
def get_j_coeff(a,b):
    if len(a) != len(b):
        return jaccard_score_inconsistent(a, b)
    return jaccard_score(a, b, average='micro')
        

def sample_paths(x,path_model=path_m,label_model=label_m,latent_dim=latent_dim,feature_size=feature_size):
    n = x.shape[0]
    x_f = x.reshape(1,feature_size)
    token = 'S'
    cont = True
    text = [token]
    x_sent = np.zeros((1,maxlen,vocab_size),dtype=np.bool)
    x_latent = get_hidden_x(x_f,model=label_model)
    x_latent = x_latent.reshape(1,latent_dim)
    x_sent[0,0,char_indices[token]] = 1
    pred = label_model.predict(x_f)
    label = [np.argmax(pred[0])]
    index = 1
    while cont & (index <maxlen):
        pred = path_model.predict([x_latent,x_sent])
        char_index = np.argmax(pred[0])
        x_sent[0,index,char_index] = 1
        next_char = indices_char[char_index]
        text.append(next_char)
        index += 1    
        if next_char == 'E':
            cont = False
    return [text,label]

def predict_decision_feature(x, path_step)

count = []
j_coeff = []
l_dist = []
for i in range(150):
    curr_feat = np.array([df.iloc[i,0:4]])
    path,label= sample_paths(curr_feat)
#     print('actual vs predicted: ', df.iloc[i,4] ,' vs ', ' '.join(path), 'labels: ', df.iloc[i,5],label[0])
    count.append(df.iloc[i,5]==label[0])
    actual_path = df.iloc[i,4].split()
    actual_path_tok = [char_indices[char] for char in actual_path]
    pred_path_tok = [char_indices[char] for char in path]
    j_coeff.append(get_j_coeff(actual_path_tok, pred_path_tok))
    l_dist.append(distance.levenshtein(df.iloc[i,4].replace(' ', ''), ''.join(path)))
print('\nLabel accuracy - ',np.mean(count))
print('Path metric (Jaccard) - ',np.mean(j_coeff))
print('Path metric (Levensthein) - ',np.mean(l_dist))

In [ ]:
from keras.utils import to_categorical

label_m.evaluate(df.iloc[:,0:4], to_categorical(df.iloc[:,5]))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(label_m_history.history['acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(label_m_history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Optimizations required

1) Path comparison - dt vs rnn output  -- Jaccard coeff/ levenstein distance (try both
2) cutpoints while training  -- feature information
   - Predict the precise cutpoint(regression model) 
   - 3 buckets(low, med, high)--range quantile splits1/3,2/3 -- Classification model
   - 4 classification models - 
3) Packaging it as a usable API  
4) latent_dim as hyperparam - ray/tune  
5) Test/train split. Validation

Tests
---

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Concatenate, concatenate, Flatten, GRU

feature_size = 4
h1_size = 5
input_x_features = Input(shape=(feature_size,),name='ip_x')
hidden_state_x1 = Dense(20, activation='tanh',name='hidden_x1')(input_x_features)
hidden_state_x2 = Dense(10, activation='tanh',name='hidden_x2')(hidden_state_x1)
hidden_state_x3 = Dense(h1_size, activation='tanh',name='hidden_x3')(hidden_state_x2)
output_labels = Dense(3, activation='softmax',name='op_x')(hidden_state_x3)    
model = Model(input_x_features,output_labels)


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X,y_cat,batch_size =30, epochs = 200,verbose=1, shuffle=True)

In [ ]:
model.evaluate(X, y_cat)

In [ ]:
## Jaccard coeff trials

from sklearn.metrics import jaccard_score

from scipy.spatial import distance
distance.jaccard([1, 0, 0, 1], [0, 1, 0])


In [ ]:
# Model to predict decision feature

np.array([[[1,0],[0,0,0,1],[0,1,0]],
          [[1,0],[0,0,0,1],[0,1,0]]]).shape

In [8]:
## Trial model -- to predict decision features
from keras.layers import Input, Dense
from keras.models import Model

input_1 = Input(shape=(5,))
hidden_1 = Dense(8, activation='tanh')(input_1)
output = Dense(5, activation='softmax')(hidden_1)

test_model = Model(input_1, output)

test_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [9]:
test_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 48        
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 45        
Total params: 93
Trainable params: 93
Non-trainable params: 0
_________________________________________________________________


In [11]:
chars = ['S', 'L', 'R', 'E']
char_indices = dict((c, i) for i, c in enumerate(chars))

for i, j in enumerate(final_test_df.values):
    final_test_df[5][i] = char_indices[final_test_df[5][i]]

/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [12]:
final_test_df = final_test_df.drop(columns=4)

trial_x = final_test_df.iloc[:,0:5]
trial_y = final_test_df.iloc[:,5]

# np.append(converted_chars, char_indices[final_test_df[5][i]])

In [13]:
final_test_df.head(20)

,0,1,2,3,5,6
0,-0.900681,1.019,-1.34023,-1.31544,0,0
1,-0.900681,1.019,-1.34023,-1.31544,1,0
2,-0.900681,1.019,-1.34023,-1.31544,3,0
3,-1.14302,-0.131979,-1.34023,-1.31544,0,0
4,-1.14302,-0.131979,-1.34023,-1.31544,1,0
5,-1.14302,-0.131979,-1.34023,-1.31544,3,0
6,-1.38535,0.328414,-1.39706,-1.31544,0,0
7,-1.38535,0.328414,-1.39706,-1.31544,1,0
8,-1.38535,0.328414,-1.39706,-1.31544,3,0
9,-1.50652,0.0982173,-1.28339,-1.31544,0,0


In [14]:
final_test_df.shape

(676, 6)

In [15]:
final_test_df[200:210]

,0,1,2,3,5,6
200,-1.02185,-2.43395,-0.146641,-0.262387,0,0
201,-1.02185,-2.43395,-0.146641,-0.262387,2,3
202,-1.02185,-2.43395,-0.146641,-0.262387,1,4
203,-1.02185,-2.43395,-0.146641,-0.262387,1,0
204,-1.02185,-2.43395,-0.146641,-0.262387,3,0
205,0.0686618,-0.131979,0.251221,0.395774,0,0
206,0.0686618,-0.131979,0.251221,0.395774,2,3
207,0.0686618,-0.131979,0.251221,0.395774,1,4
208,0.0686618,-0.131979,0.251221,0.395774,1,0
209,0.0686618,-0.131979,0.251221,0.395774,3,0


In [16]:
from keras.utils import to_categorical

trial_y = to_categorical(trial_y)

In [17]:
trial_y.shape

(676, 5)

In [18]:
test_model.fit(trial_x, trial_y, batch_size =30, epochs = 2,verbose=1, shuffle=True)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
676/676 [==============================] - 2s 3ms/step - loss: 1.7179 - acc: 0.0740
Epoch 2/2
676/676 [==============================] - 0s 29us/step - loss: 1.6204 - acc: 0.1746
